In [1]:
from streaming.base import MDSWriter
from tqdm import tqdm

from ldm.data.hpa23 import HPA

In [2]:
dataset = HPA(include_densenet_embedding="all", include_seq_embedding="prott5-swissprot")
dataset[0]

Finding images of the same protein: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 73054/73054 [00:01<00:00, 39333.64it/s]


Serializing 73054 elements to byte tensors and concatenating them all ...
Serialized dataset takes 1.15 MiB
Dataset group: train, length: 73054, image channels: [1 1 1]


{'hpa_index': 373348,
 'bbox_label': 1,
 'condition_caption': 'NOX3/SH-SY5Y',
 'location_caption': 'nan',
 'matched_location_classes': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1.], dtype=float32),
 'image': array([[[-1.       , -1.       , -1.       ],
         [-1.       , -1.       , -1.       ],
         [-1.       , -1.       , -1.       ],
         ...,
         [-1.       , -1.       , -1.       ],
         [-1.       , -1.       , -1.       ],
         [-1.       , -1.       , -1.       ]],
 
        [[-1.       , -1.       , -1.       ],
         [-1.       , -1.       , -1.       ],
         [-1.       , -1.       , -1.       ],
         ...,
         [-1.       , -1.       , -1.       ],
         [-1.       , -1.       , -1.       ],
         [-1.       , -1.       , -1.       ]],
 
        [[-1.       , -1.       , -1.       ],
         [-1.       , -1.       , -1.       ],
         [-1.       , -1.       , -1.       ],
         

In [3]:
{k:type(v) for k,v in dataset[0].items()}

{'hpa_index': numpy.int64,
 'bbox_label': numpy.int64,
 'condition_caption': str,
 'location_caption': str,
 'matched_location_classes': numpy.ndarray,
 'image': numpy.ndarray,
 'ref-image': numpy.ndarray,
 'mask': numpy.ndarray,
 'bbox_coords': numpy.ndarray,
 'seq_embed': numpy.ndarray,
 'densent_avg': numpy.ndarray}

In [4]:
# Local or remote directory path to store the output compressed files.
# For remote directory, the output files are automatically upload to a remote cloud storage
# location.
out_root = 's3://ai-residency-stanford-subcellgenai/super_multiplex_cell/data/hpa23_rescaled_mds'

# A dictionary of input fields to an Encoder/Decoder type
columns = {'hpa_index': "int64",
 'bbox_label': "int64",
 'condition_caption': "str",
 'location_caption': "str",
 'matched_location_classes': "ndarray",
 'image': "ndarray",
 'ref-image': "ndarray",
 'mask': "ndarray",
 'bbox_coords': "ndarray",
 'densent_avg': "ndarray",
 'seq_embed': "ndarray"}

# Compression algorithm name
compression = 'zstd'

# Hash algorithm name
hashes = 'sha1', 'xxh64'

# Call `MDSWriter` to iterate through the input data and write into a shard `mds` file
c = 0
with MDSWriter(out=f"{out_root}/train_subset", columns=columns, compression=compression, hashes=hashes) as out:
    for sample in tqdm(dataset):
        out.write(sample)
        c += 1
        if c >= 10:
            break

  0%|                                                                                                                                              | 9/1533711 [00:00<22:19:41, 19.08it/s]


In [ ]:
import pdb; pdb.pm()

> /home/users/xikunz2/stable-diffusion/ldm/data/image_processing.py(112)get_bbox_from_mask()
    110 
    111     # Make sure the object is not in the mask, return None or handle as needed
--> 112     assert len(positions[0]) > 0 and len(positions[1]) > 0
    113 
    114     # Calculate bounding box



ipdb>  positions


(array([], dtype=int64), array([], dtype=int64))


ipdb>  mask


array([[3, 3, 3, ..., 0, 0, 0],
       [3, 3, 3, ..., 0, 0, 0],
       [3, 3, 3, ..., 0, 0, 0],
       ...,
       [3, 3, 3, ..., 4, 4, 4],
       [3, 3, 3, ..., 4, 4, 4],
       [3, 3, 3, ..., 4, 4, 4]], dtype=uint8)


...

5


ipdb>  l


    107 def get_bbox_from_mask(mask, bbox_label):
    108     # Identify the positions of the object in the mask
    109     positions = np.where(mask == bbox_label)
    110 
    111     # Make sure the object is not in the mask, return None or handle as needed
--> 112     assert len(positions[0]) > 0 and len(positions[1]) > 0
    113 
    114     # Calculate bounding box
    115     top = np.min(positions[0])
    116     left = np.min(positions[1])
    117     bottom = np.max(positions[0])



ipdb>  u


> /home/users/xikunz2/stable-diffusion/ldm/data/hpa23.py(156)__getitem__()
    154         ref_image = imarray[:, :, [0, 3, 2]] # reference channels: MT, ER, DAPI
    155         ref_image = image_processing.convert_to_minus1_1(ref_image)
--> 156         top, left, bottom, right = image_processing.get_bbox_from_mask(maskarray, bbox_label)
    157         bbox_coords = np.array([top, left, bottom - top, right - left])
    158         sample.update({"image": prot_image, "ref-image": ref_image, "mask": maskarray, "bbox_coords": bbox_coords})

